### Important!!

The instructions for this notebook are slightly different.

First run the below cell (starts with "!pip install").

Then, click "Runtime" -> "Restart and Run All", inputting files when prompted.



In [1]:
!pip install --upgrade pandas==1.2

     |████████████████████████████████| 9.9 MB 2.7 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.2.0 which is incompatible.


# Topic Modeling

In [ ]:
import pandas as pd
import seaborn as sns
#!pip install -U pandas-profiling

from google.colab import files

# Directions

The below cells will each prompt you for the a file upload. The first file should be in the same format as the `nps_comments.csv` file shared with our team. The second should be in the same format as `product_reviews.csv`, and the thirs should be in the same format as `subscription_cancellations.csv`.

There is also a table of contents on the left sidebar to help with navigation!

In [ ]:
uploaded_nps = files.upload()

Saving nps_comments.csv to nps_comments (2).csv


In [ ]:
uploaded_pr = files.upload()

Saving product_reviews.csv to product_reviews (2).csv


In [ ]:
uploaded_sc = files.upload()

Saving subscription_cancellations.csv to subscription_cancellations (2).csv


In [ ]:
import io
comments = pd.read_csv(io.BytesIO(uploaded_nps['nps_comments.csv']))
reviews = pd.read_csv(io.BytesIO(uploaded_pr['product_reviews.csv']))
cancellations = pd.read_csv(io.BytesIO(uploaded_sc['subscription_cancellations.csv']))

**1. Data Preprocessing**

> Main Focus: Reason as to why people cancel their orders.



In [ ]:
#find all unique (possible) reasons for cancelling subscription
cancellations['cancellation_reason'].unique()

array(['Other reason',
       'Prefer to order refills as needed (without a subscription)',
       'I no longer use this product',
       "The product/formula didn't work for me", 'Other',
       'This is too expensive', 'This was created by accident',
       "Product wasn't a good value ", "The product didn't work for me",
       "The container didn't work for me",
       'Just not that into subscriptions',
       "I didn't like the scent/flavor", "The formula didn't work for me",
       "I'm not ready for my next refill (still have plenty)",
       "Product wasn't a good value", 'I need it sooner'], dtype=object)

In [ ]:
cancellations.describe()

,id
count,8.086000e+03
mean,7.589523e+07
std,2.722157e+07
min,3.304312e+07
25%,5.258245e+07
50%,6.859638e+07
75%,9.613456e+07
max,1.471388e+08


In [ ]:
count1 = cancellations['cancellation_reason'].value_counts()

In [ ]:
count1

Other reason                                                  2649
The product didn't work for me                                1415
The product/formula didn't work for me                        1251
Prefer to order refills as needed (without a subscription)     552
I'm not ready for my next refill (still have plenty)           490
I didn't like the scent/flavor                                 451
Other                                                          357
The formula didn't work for me                                 342
The container didn't work for me                               241
Product wasn't a good value                                    235
Product wasn't a good value                                     57
I no longer use this product                                    29
This is too expensive                                            7
Just not that into subscriptions                                 6
This was created by accident                                  

In [ ]:
reviews["RATING"].value_counts()

Great       673
Ok          278
Good        202
Critical    145
Name: RATING, dtype: int64

In [ ]:
reviews["PRODUCT"].value_counts()

Deodorant          528
Shampoo Bar        185
Mouthwash          181
Floss              160
Hand Sanitizer      71
Toothpaste          68
Conditioner Bar     60
Body Wash Bar       22
Cotton Swabs        12
Hand Soap Bar        8
Travel Sets          2
Dopp Kit             1
Name: PRODUCT, dtype: int64

In [ ]:
reviews[reviews["PRODUCT"] == "Deodorant"]

,PRODUCT,RATING,REVIEW,DATE
1,Deodorant,Great,Best natural deodorant I have ever used. Defin...,3/24/20
4,Deodorant,Great,Great product! Love the single use container e...,4/6/20
5,Deodorant,Great,"Honestly, the first natural deodorant I’ve tri...",2/18/20
7,Deodorant,Great,I love everything about this deodorant. The tu...,2/21/20
8,Deodorant,Great,AMAZING natural deodorant. I have been using n...,4/5/20
...,...,...,...,...
1292,Deodorant,Great,I’ve tried at least 6 different natural deodor...,4/12/21
1294,Deodorant,Critical,I bought the deodorant for my daughter and it ...,4/12/21
1295,Deodorant,Great,"I LOVED the packaging, the smell and the fact ...",4/13/21
1296,Deodorant,Great,It works well and smells great,4/14/21


## **NLP Preprocessing text**
source: https://towardsdatascience.com/introduction-to-nlp-part-1-preprocessing-text-in-python-8f007d44ca96

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#15visualizethetopicskeywords
"LDA"

In [ ]:
import nltk
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Run in python console
import nltk; nltk.download('stopwords')

# $ pip install -U pip setuptools wheel
# $ pip install -U spacy
# $ python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#!pip install --upgrade pandas==1.2

!pip install pyLDAvis==2.1.2
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
!pip install gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis;
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
#Dataset
cancellations.head()

,id,cancelled_at,product_name,cancellation_reason,cancellation_reason_comments
0,36179160,2019-10-19 21:17:53+00,Deodorant,Other reason,I found another product that I prefer
1,36180516,2019-07-07 15:53:42+00,Deodorant,Prefer to order refills as needed (without a s...,thanks
2,36184623,2019-02-28 11:37:18+00,Deodorant,I no longer use this product,Not satisfied with product
3,36188379,2019-03-06 10:56:11+00,Deodorant,Other reason,I don't go through this much product.
4,36225477,2019-02-20 10:40:43+00,Deodorant,Other reason,Deoderant didn't fit in the plastic holder. Cr...


In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

cancellations.head()

,id,cancelled_at,product_name,cancellation_reason,cancellation_reason_comments
0,36179160,2019-10-19 21:17:53+00,Deodorant,Other reason,I found another product that I prefer
1,36180516,2019-07-07 15:53:42+00,Deodorant,Prefer to order refills as needed (without a s...,thanks
2,36184623,2019-02-28 11:37:18+00,Deodorant,I no longer use this product,Not satisfied with product
3,36188379,2019-03-06 10:56:11+00,Deodorant,Other reason,I don't go through this much product.
4,36225477,2019-02-20 10:40:43+00,Deodorant,Other reason,Deoderant didn't fit in the plastic holder. Cr...


In [ ]:
print(cancellations.cancellation_reason.unique())

['Other reason'
 'Prefer to order refills as needed (without a subscription)'
 'I no longer use this product' "The product/formula didn't work for me"
 'Other' 'This is too expensive' 'This was created by accident'
 "Product wasn't a good value " "The product didn't work for me"
 "The container didn't work for me" 'Just not that into subscriptions'
 "I didn't like the scent/flavor" "The formula didn't work for me"
 "I'm not ready for my next refill (still have plenty)"
 "Product wasn't a good value" 'I need it sooner']


In [ ]:
data = cancellations.cancellation_reason_comments.values.tolist()

In [ ]:
#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['found', 'another', 'product', 'that', 'prefer']]


In [ ]:
#Creating Bigram and Trigram Models
#Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['found', 'another', 'product', 'that', 'prefer']


In [ ]:
#Remove Stopwords, Make Bigrams and Lemmatize

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

Warning: this cell may take a long time to run!

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['find', 'product', 'prefer']]


In [ ]:
#create dictionary & corpus for topic modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('find', 1), ('prefer', 1), ('product', 1)]]

## Building the Topic Model
"chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes"

citations: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.142*"really" + 0.119*"would" + 0.070*"tablet" + 0.061*"scent" + '
  '0.060*"flavor" + 0.059*"also" + 0.040*"prefer" + 0.025*"apply" + '
  '0.021*"formula" + 0.020*"free"'),
 (1,
  '0.382*"product" + 0.106*"much" + 0.060*"find" + 0.054*"good" + '
  '0.030*"brand" + 0.020*"effective" + 0.017*"shower" + 0.015*"armpit" + '
  '0.015*"better" + 0.014*"terrible"'),
 (2,
  '0.208*"tooth" + 0.103*"hard" + 0.062*"pull" + 0.041*"stuck" + 0.033*"see" + '
  '0.033*"husband" + 0.029*"able" + 0.024*"type" + 0.024*"however" + '
  '0.023*"cause"'),
 (3,
  '0.194*"want" + 0.155*"enough" + 0.104*"toothpaste" + 0.051*"strong" + '
  '0.049*"week" + 0.033*"change" + 0.032*"may" + 0.027*"weird" + 0.019*"foam" '
  '+ 0.017*"next"'),
 (4,
  '0.248*"make" + 0.079*"taste" + 0.074*"mouthwash" + 0.073*"wash" + '
  '0.037*"waste" + 0.035*"water" + 0.033*"rinse" + 0.030*"ship" + '
  '0.024*"worth" + 0.020*"package"'),
 (5,
  '0.127*"get" + 0.086*"refill" + 0.076*"time" + 0.068*"go" + 0.045*"dry" + '
  '0.0

In [ ]:
#model complexity & coherence score
#measure to judge how good a given topic model is
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.05184340234637

Coherence Score:  0.4282824900746073


## Visualize Topic Keywords

In [ ]:
#!pip install --upgrade pandas==1.2

In [ ]:
#!pip install --upgrade pandas

In [ ]:
# import pyLDAvis
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis

#visualize the topics
#!pip install --upgrade pandas==1.2
#!pip install pyldavis==2.1.0
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

#interpret:
#Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

#A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

#A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

#Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)
2021-08-29 22:08:39,796 : ERROR : exception calling callback for <Future at 0x7f1779a3f190 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'new_block' on <module 'pandas.core.internals.blocks' from '/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py'>
"""

The above exception was the direct cause of the following exception:

Traceback (

BrokenProcessPool: ignored